In [1]:
import numpy as np
np.random.seed(2016)
import scipy
import os
import glob
import math
import pickle
import datetime
#import pandas as pd

from keras.applications import vgg16 as keras_vgg16
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D,Conv2D
from keras.layers import Activation, Dropout, Flatten, Dense ,Input, BatchNormalization, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import RMSprop
from keras.optimizers import Adam
from keras.layers.core import  ActivityRegularization

import json, sys

import keras
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.preprocessing import image

import matplotlib.pyplot as plt

F:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
w_img, h_img = 224, 224

training_path = 'train'
validation_path = 'validation'
testing_path = 'test'

classes = 7;
BATCH_SIZE = 16;
EPOCHS_NB = 1;

input_shape = (224,224,3)

num_train_samples = sum([len(files) for r, d, files in os.walk(training_path)])
num_valid_samples = sum([len(files) for r, d, files in os.walk(validation_path)])
num_test_samples = sum([len(files) for r, d, files in os.walk(testing_path)])

print(num_train_samples)
print(num_valid_samples)
print(num_test_samples)

num_train_steps = math.floor(num_train_samples/BATCH_SIZE)
num_valid_steps = math.floor(num_valid_samples/BATCH_SIZE)

print(num_train_steps)
print(num_valid_steps)

902
432
147
56
27


In [39]:
train_batches = ImageDataGenerator(rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1, 
        height_shift_range=0.1,
        shear_range=0.15,
        zoom_range=0.1, 
        channel_shift_range=10.,
        horizontal_flip=True).flow_from_directory(training_path, target_size=(w_img,h_img), batch_size=BATCH_SIZE)



"""
train_batches = ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True).flow_from_directory(training_path, target_size=(w_img,h_img), batch_size=BATCH_SIZE)
"""
valid_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(validation_path, target_size=(w_img,h_img), batch_size=BATCH_SIZE)
test_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(testing_path, target_size=(w_img,h_img), batch_size=4)

Found 902 images belonging to 7 classes.
Found 432 images belonging to 7 classes.
Found 147 images belonging to 7 classes.


In [40]:
base_modelResNet = keras.applications.resnet50.ResNet50(include_top=False,  weights='imagenet')

In [41]:
base_modelResNet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        input_2[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_50 (Activation)      (None, None, None, 6 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [42]:
lastLayer = base_modelResNet.output
x = Conv2D(1024, (3, 3), name='convNew', padding='same')(lastLayer)
#x = BatchNormalization(axis=-1, name='banNew')(x)
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='elu', name='fc-1')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='elu', name='fc-2')(x)
x = Dropout(0.5)(x)
x = ActivityRegularization(l2=0.01)(x)
output = Dense(classes, activation='softmax', name='outputLayer')(x)

customModel = Model(inputs=base_modelResNet.input, outputs=output)

customModel.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        input_2[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_50 (Activation)      (None, None, None, 6 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [43]:
for layer in customModel.layers[:-8]:
        layer.trainable=False
        
#customModel.layers[-1].trainable

In [44]:
#rmsProp =RMSprop( lr=0.0001)
adm = Adam(lr=0.0001)

#customModel.compile(optimizer=rmsProp, loss='categorical_crossentropy', metrics=['accuracy'])
customModel.compile(optimizer=adm, loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
# Model Checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkPoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkPoint]

In [46]:
customModel.fit_generator(
        train_batches,
        steps_per_epoch=num_train_steps,
        epochs=EPOCHS_NB,
        validation_data=valid_batches,
        validation_steps=num_valid_steps,
        callbacks=callbacks_list)

Epoch 1/1
 2/56 [>.............................] - ETA: 35:06 - loss: 10.7586 - acc: 0.1562    

KeyboardInterrupt: 

In [ ]:

score = customModel.evaluate_generator(test_batches, num_test_samples)
print("Accuracy = ", score[1])

customModel.save_weights('Weights_v1.h5',overwrite=True)

In [ ]:
#References
#https://www.youtube.com/watch?v=m5RjXjvAAhQ
#https://www.youtube.com/watch?v=14syUbL16k4